In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('./datasets/train.csv', low_memory=False)

# Display the first few rows of the dataframe to understand its structure
print(data.head())


       id Tranc_YearMonth             town  flat_type block       street_name  \
0   88471         2016-05  KALLANG/WHAMPOA     4 ROOM    3B  UPP BOON KENG RD   
1  122598         2012-07           BISHAN     5 ROOM   153      BISHAN ST 13   
2  170897         2013-07      BUKIT BATOK  EXECUTIVE  289B    BT BATOK ST 25   
3   86070         2012-04           BISHAN     4 ROOM   232      BISHAN ST 22   
4  153632         2017-12           YISHUN     4 ROOM   876      YISHUN ST 81   

  storey_range  floor_area_sqm  flat_model  lease_commence_date  ...  vacancy  \
0     10 TO 12            90.0     Model A                 2006  ...       78   
1     07 TO 09           130.0    Improved                 1987  ...       45   
2     13 TO 15           144.0   Apartment                 1997  ...       39   
3     01 TO 05           103.0     Model A                 1992  ...       20   
4     01 TO 03            83.0  Simplified                 1987  ...       74   

   pri_sch_affiliation  pr

In [2]:
# Check for missing values in each column
missing_values = data.isnull().sum()

# Percentage of missing values
missing_percentage = (missing_values / len(data)) * 100

# Combine the missing values count and percentage into a DataFrame for better visualization
missing_df = pd.DataFrame({'missing_values': missing_values, 'percentage': missing_percentage})

# Display columns with missing values sorted by percentage of missing data
missing_data_filtered = missing_df[missing_df['missing_values'] > 0].sort_values(by='percentage', ascending=False)


In [3]:
from sklearn.impute import SimpleImputer

# Dropping columns with a high percentage of missing values
columns_to_drop = missing_data_filtered[missing_data_filtered['percentage'] > 40].index
data_cleaned = data.drop(columns=columns_to_drop)

# Imputing missing values for columns with lower missing percentages
# Since all these columns are related to distances, we'll use a median strategy for imputation
imputer = SimpleImputer(strategy='median')
columns_to_impute = missing_data_filtered[(missing_data_filtered['percentage'] <= 40) & (missing_data_filtered['percentage'] > 0)].index
data_cleaned[columns_to_impute] = imputer.fit_transform(data_cleaned[columns_to_impute])

# Feature Engineering: Calculating flat_age
# Convert Tranc_YearMonth to datetime to extract the year, then subtract lease_commence_date from the transaction year
data_cleaned['Tranc_YearMonth'] = pd.to_datetime(data_cleaned['Tranc_YearMonth'])
data_cleaned['transaction_year'] = data_cleaned['Tranc_YearMonth'].dt.year
data_cleaned['flat_age'] = data_cleaned['transaction_year'] - data_cleaned['lease_commence_date']
data_cleaned = pd.get_dummies(data_cleaned, columns=['flat_type'], drop_first=True)

# Dropping the now redundant or temporary columns used for feature engineering
data_cleaned.drop(columns=['Tranc_YearMonth', 'transaction_year'], inplace=True)

# Checking the dataset after cleaning and feature engineering
data_cleaned.head()


,id,town,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Tranc_Year,...,affiliation,sec_sch_latitude,sec_sch_longitude,flat_age,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION
0,88471,KALLANG/WHAMPOA,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,680000.0,2016,...,0,1.317659,103.882504,10,False,False,True,False,False,False
1,122598,BISHAN,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,665000.0,2012,...,0,1.350110,103.854892,25,False,False,False,True,False,False
2,170897,BUKIT BATOK,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,838000.0,2013,...,0,1.342334,103.760013,16,False,False,False,False,True,False
3,86070,BISHAN,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,550000.0,2012,...,1,1.354789,103.844934,20,False,False,True,False,False,False
4,153632,YISHUN,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,298000.0,2017,...,0,1.414888,103.838335,30,False,False,True,False,False,False


In [4]:
from math import radians, cos, sin, sqrt, atan2
import numpy as np
# Define the coordinates for the Central Business District (CBD)
cbd_coordinates = (1.2831, 103.8511)

# Define a function to calculate distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Difference in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance

# Recalculate distances using the Haversine formula
data_cleaned.loc[:,'dist_from_cbd_km'] = data_cleaned.apply(
    lambda row: haversine(row['Latitude'], row['Longitude'], cbd_coordinates[0], cbd_coordinates[1]),
    axis=1
)

# Display the first few rows to confirm the new distance calculations
data_cleaned[['town', 'Latitude', 'Longitude', 'dist_from_cbd_km']].head()

,town,Latitude,Longitude,dist_from_cbd_km
0,KALLANG/WHAMPOA,1.314299,103.872828,4.227211
1,BISHAN,1.346086,103.855078,7.017668
2,BUKIT BATOK,1.343867,103.760535,12.125001
3,BISHAN,1.358245,103.845504,8.378885
4,YISHUN,1.414745,103.835532,14.740182


In [5]:
# Import and read dataset
ballot_df = pd.read_csv('./datasets/p1_ballot_2021.csv')

# Clean the school names by removing special characters and trimming spaces
ballot_df['School'] = ballot_df['School'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()

# Calculate the popularity metric as the ratio of Applications to Vacancies
ballot_df['Popularity_Ratio'] = ballot_df['Applications'] / ballot_df['Vacancies']

# Determine the quantile values for splitting the tiers
quantiles = ballot_df['Popularity_Ratio'].quantile([1/3, 2/3])

# Assign tiers based on quantiles
ballot_df['pri_sch_tier'] = pd.cut(ballot_df['Popularity_Ratio'],
                              bins=[0, quantiles.iloc[0], quantiles.iloc[1], ballot_df['Popularity_Ratio'].max()],
                              labels=['pri_sch_tier_3', 'pri_sch_tier_2', 'pri_sch_tier_1'],
                              include_lowest=True)

# If you want to view the unique cleaned school names:
unique_schools = ballot_df['School'].unique()
print(sorted(unique_schools))  # This will print sorted unique school names

# Assuming data_cleaned exists and you want to clean primary school names within it
data_cleaned['pri_sch_name'] = data_cleaned['pri_sch_name'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()

['Admiralty', 'Ahmad Ibrahim', 'Ai Tong', 'Alexandra', 'Anchor Green', 'Anderson', 'Ang Mo Kio', 'AngloChinese Junior', 'AngloChinese Primary', 'Angsana', 'Beacon', 'Bedok Green', 'Bendemeer', 'Blangah Rise', 'Boon Lay Garden', 'Bukit Panjang', 'Bukit Timah', 'Bukit View', 'CHIJ Katong', 'CHIJ Kellock', 'CHIJ Our Lady Queen of Peace', 'CHIJ Our Lady of Good Counsel', 'CHIJ Our Lady of the Nativity', 'CHIJ St Nicholas Girls', 'CHIJ Toa Payoh', 'Canberra', 'Canossa Catholic', 'Cantonment', 'Casuarina', 'Catholic High', 'Cedar', 'Changkat', 'Chongfu', 'Chongzheng', 'Chua Chu Kang', 'Clementi', 'Compassvale', 'Concord', 'Corporation', 'Damai', 'Dazhong', 'De La Salle', 'East Spring', 'Edgefield', 'Elias Park', 'Endeavour', 'Evergreen', 'Fairfield Methodist', 'Farrer Park', 'Fengshan', 'Fern Green', 'Fernvale', 'First Toa Payoh', 'Frontier', 'Fuchun', 'Fuhua', 'Gan Eng Seng', 'Geylang Methodist', 'Gongshang', 'Greendale', 'Greenridge', 'Greenwood', 'Haig Girls', 'Henry Park', 'Holy Innocent

In [6]:
from fuzzywuzzy import process

# Define a function to perform fuzzy matching and return the best match and its score
def fuzzy_match_school(x, choices, scorer, cutoff):
    match = process.extractOne(x, choices, scorer=scorer, score_cutoff=cutoff)
    return match if match else (None, None)

# School choices from the original dataset
school_choices = ballot_df['School'].unique()

# Prepare lists of unique school names from both datasets for fuzzy matching
unique_new_school_names = data_cleaned['pri_sch_name'].unique()
unique_original_school_names = ballot_df['School'].unique()


# Adjust the fuzzy matching process to properly handle cases where no match meets the score cutoff
match_mapping = {}
for school in unique_new_school_names:
    result = process.extractOne(school, unique_original_school_names, score_cutoff=90)
    if result:
        match, score = result
        match_mapping[school] = match

# Apply the match results to the new dataset
data_cleaned['pri_sch_name_matched'] = data_cleaned['pri_sch_name'].map(match_mapping).fillna(data_cleaned['pri_sch_name'])

# Apply fuzzy matching to all school names in the new dataset with a score cutoff to filter for scores less than 90
all_school_matches = {school: process.extractOne(school, school_choices, score_cutoff=0) for school in data_cleaned['pri_sch_name'].unique()}

# Filter matches to only include those with scores less than 90
matches_less_than_90 = {school: match for school, match in all_school_matches.items() if match and match[1] < 90}

# Correction mapping
correction_mapping = {
    "Saint Margarets Primary School": "St Margarets",
    "Saint Andrews Junior School": "St Andrews Junior",
    "CHIJ Saint Nicholas Girls School": "CHIJ St Nicholas Girls",
    "Saint Anthonys Canossian Primary School": "St Anthonys Canossian",
}

# Applying corrections to the 'pri_sch_name' column
data_cleaned['pri_sch_name'] = data_cleaned['pri_sch_name'].apply(lambda x: correction_mapping.get(x, x))


# Merge the tier information from the original dataset into the new dataset based on the matched school names
data_cleaned = pd.merge(data_cleaned, ballot_df[['School', 'pri_sch_tier']], left_on='pri_sch_name_matched', right_on='School', how='left')

# Display a few rows of the merged dataset to verify the tier information has been correctly merged
data_cleaned[['pri_sch_name', 'pri_sch_name_matched', 'School', 'pri_sch_tier']].sample(10)

# Drop rows with NaN values in the 'pri_sch_tier' column to clean the dataset
data_cleaned = data_cleaned.dropna(subset=['pri_sch_tier'])

# Perform one-hot encoding on the 'pri_sch_tier' column, dropping the first category to avoid multicollinearity
pri_sch_tier_one_hot = pd.get_dummies(data_cleaned['pri_sch_tier'], drop_first=True)

# Reset the index of both DataFrames if they might not be aligned or to ensure compatibility
data_cleaned_reset = data_cleaned.reset_index(drop=True)
pri_sch_tier_one_hot_reset = pri_sch_tier_one_hot.reset_index(drop=True)

# Ensure pri_sch_tier_one_hot' matches the number of rows in 'data_cleaned_reset'
# Concatenate them column-wise
data_cleaned = pd.concat([data_cleaned_reset, pri_sch_tier_one_hot_reset], axis=1)
print(data_cleaned.sample(10))

# Drop the specified columns from the merged data
data_cleaned = data_cleaned.drop(columns=['pri_sch_name_matched', 'School', 'pri_sch_tier'])

# Convert the one-hot encoded tier columns to integers
tier_columns = ['pri_sch_tier_2', 'pri_sch_tier_1']
data_cleaned[tier_columns] = data_cleaned[tier_columns].astype(int)

# Verify the data types of the tier columns and display a few rows to confirm
data_cleaned[tier_columns].dtypes, data_cleaned[['pri_sch_name', *tier_columns]].sample(10)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


            id             town block       street_name storey_range  \
69599    32881          HOUGANG   688     HOUGANG ST 61     04 TO 06   
60543    75715        SERANGOON   421    SERANGOON CTRL     04 TO 06   
46546    95866  KALLANG/WHAMPOA    19      JLN TENTERAM     16 TO 18   
83073    25906    CHOA CHU KANG     1     TECK WHYE AVE     07 TO 09   
130097   15164        WOODLANDS    22      MARSILING DR     04 TO 06   
108648   44621           YISHUN   321       YISHUN CTRL     10 TO 12   
74231   124777         SENGKANG  267B  COMPASSVALE LINK     07 TO 09   
11507   136370          PUNGGOL  604A        PUNGGOL RD     04 TO 06   
114821  122872      BUKIT MERAH    6B     BOON TIONG RD     19 TO 21   
91310    95035          HOUGANG   411    HOUGANG AVE 10     04 TO 06   

        floor_area_sqm         flat_model  lease_commence_date  resale_price  \
69599            103.0            Model A                 1987      348000.0   
60543            105.0            Model A      

(pri_sch_tier_2    int64
 pri_sch_tier_1    int64
 dtype: object,
                      pri_sch_name  pri_sch_tier_2  pri_sch_tier_1
 43688       Keming Primary School               0               1
 130781      Huamin Primary School               0               1
 52289   Elias Park Primary School               1               0
 9206        Xishan Primary School               0               1
 27737       Rulang Primary School               0               1
 55605    Evergreen Primary School               0               0
 111544     Punggol Primary School               0               1
 99240      Lianhua Primary School               0               0
 35233    Greendale Primary School               0               0
 56113      Zhangde Primary School               1               0)

In [7]:

columns_to_drop = ['address', 'postal', 'block', 'bus_stop_name', 'street_name', 
                   'pri_sch_name', 'sec_sch_name', 'mrt_name', 'id', 'Tranc_Year', 'Tranc_Month', 'mid_storey', 
                   'lower', 'upper', 'mid', 'floor_area_sqft', 'price_per_sqft', 'max_floor_lvl', 
                   'year_completed', 'lease_commence_date', 'total_dwelling_units', 
                   '1room_rental', '2room_rental', '3room_rental', 
                   'other_room_rental', 'Latitude', 'Longitude', 
                   'mrt_latitude', 'mrt_longitude', 
                   'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_nearest_distance', 
                   'pri_sch_affiliation', 'pri_sch_latitude', 'pri_sch_longitude', 
                   'sec_sch_nearest_dist', 'cutoff_point', 'affiliation', 'sec_sch_latitude', 
                   'sec_sch_longitude', 'multistorey_carpark', 'full_flat_type', 'commercial', 
                   'precinct_pavilion', 'residential', 'storey_range', 'planning_area', 'hdb_age', 'vacancy', 
                   'exec_sold', '1room_sold', 'mrt_nearest_distance', 'bus_stop_nearest_distance', 
                   'studio_apartment_sold', 'hawker_food_stalls', 'hawker_market_stalls', 'flat_model', 
                   'Mall_Nearest_Distance', '2room_sold', '3room_sold', 
                   '4room_sold', '5room_sold', 'multigen_sold', 'Hawker_Nearest_Distance', 'Hawker_Within_2km', 
                   'market_hawker', 'bus_interchange', 'mrt_interchange', 'flat_type_2 ROOM', 'town']
data_cleaned = data_cleaned.drop(columns=columns_to_drop)
print(data_cleaned.columns)


Index(['floor_area_sqm', 'resale_price', 'Mall_Within_1km', 'Mall_Within_2km',
       'flat_age', 'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM',
       'flat_type_EXECUTIVE', 'flat_type_MULTI-GENERATION', 'dist_from_cbd_km',
       'pri_sch_tier_2', 'pri_sch_tier_1'],
      dtype='object')


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Assuming `data_cleaned` now contains only the features you intend to keep
# Identify numerical and categorical columns excluding the target variable 'resale_price'

numerical_cols = data_cleaned.select_dtypes(include=['int64', 'float64']).columns.drop('resale_price')


# Define preprocessing for numerical columns (scale them)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        
    ])

# Define model
model = LinearRegression()

# Create and fit the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

y = data_cleaned['resale_price']
X = data_cleaned.drop(['resale_price'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)




Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['floor_area_sqm', 'Mall_Within_1km', 'Mall_Within_2km', 'flat_age',
       'dist_from_cbd_km', 'pri_sch_tier_2', 'pri_sch_tier_1'],
      dtype='object'))])),
                ('model', LinearRegression())])

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Absolute Error: 55414.41342931061
Mean Squared Error: 5276458810.874396
R-squared: 0.7441838415621083


In [10]:
# Example: Predicting on a new sample

new_data = pd.read_csv('./datasets/test.csv', low_memory=False)

new_data['Tranc_YearMonth'] = pd.to_datetime(new_data['Tranc_YearMonth'])
new_data['transaction_year'] = new_data['Tranc_YearMonth'].dt.year
new_data['flat_age'] = new_data['transaction_year'] - new_data['lease_commence_date']
new_data = pd.get_dummies(new_data, columns=['flat_type'], drop_first=True)
new_data['pri_sch_name'] = new_data['pri_sch_name'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()


# Prepare lists of unique school names from both datasets for fuzzy matching
unique_new_school_names = new_data['pri_sch_name'].unique()
unique_original_school_names = ballot_df['School'].unique()


# Adjust the fuzzy matching process to properly handle cases where no match meets the score cutoff
match_mapping = {}
for school in unique_new_school_names:
    result = process.extractOne(school, unique_original_school_names, score_cutoff=90)
    if result:
        match, score = result
        match_mapping[school] = match

# Apply the match results to the new dataset
new_data['pri_sch_name_matched'] = new_data['pri_sch_name'].map(match_mapping).fillna(new_data['pri_sch_name'])

# Apply fuzzy matching to all school names in the new dataset with a score cutoff to filter for scores less than 90
all_school_matches = {school: process.extractOne(school, school_choices, score_cutoff=0) for school in new_data['pri_sch_name'].unique()}

# Filter matches to only include those with scores less than 90
matches_less_than_90 = {school: match for school, match in all_school_matches.items() if match and match[1] < 90}

# Correction mapping
correction_mapping = {
    "Saint Margarets Primary School": "St Margarets",
    "Saint Andrews Junior School": "St Andrews Junior",
    "CHIJ Saint Nicholas Girls School": "CHIJ St Nicholas Girls",
    "Saint Anthonys Canossian Primary School": "St Anthonys Canossian",
}

# Applying corrections to the 'pri_sch_name' column
new_data['pri_sch_name'] = new_data['pri_sch_name'].apply(lambda x: correction_mapping.get(x, x))


# Merge the tier information from the original dataset into the new dataset based on the matched school names
new_data = pd.merge(new_data, ballot_df[['School', 'pri_sch_tier']], left_on='pri_sch_name_matched', right_on='School', how='left')

# Display a few rows of the merged dataset to verify the tier information has been correctly merged
new_data[['pri_sch_name', 'pri_sch_name_matched', 'School', 'pri_sch_tier']].sample(10)

# Drop rows with NaN values in the 'pri_sch_tier' column to clean the dataset
new_data = new_data.dropna(subset=['pri_sch_tier'])

# Perform one-hot encoding on the 'pri_sch_tier' column, dropping the first category to avoid multicollinearity
pri_sch_tier_one_hot = pd.get_dummies(new_data['pri_sch_tier'], drop_first=True)

# Reset the index of both DataFrames if they might not be aligned or to ensure compatibility
new_data_reset = new_data.reset_index(drop=True)
pri_sch_tier_one_hot_reset = pri_sch_tier_one_hot.reset_index(drop=True)

# Ensure pri_sch_tier_one_hot' matches the number of rows in 'new_data_reset'
# Concatenate them column-wise
new_data = pd.concat([new_data_reset, pri_sch_tier_one_hot_reset], axis=1)
print(new_data.sample(10))

# Drop the specified columns from the merged data
new_data = new_data.drop(columns=['pri_sch_name_matched', 'School', 'pri_sch_tier'])

# Convert the one-hot encoded tier columns to integers
tier_columns = ['pri_sch_tier_2', 'pri_sch_tier_1']
new_data[tier_columns] = new_data[tier_columns].astype(int)

# Verify the data types of the tier columns and display a few rows to confirm
new_data[tier_columns].dtypes, new_data[['pri_sch_name', *tier_columns]].sample(10)


# Recalculate distances using the Haversine formula
new_data.loc[:,'dist_from_cbd_km'] = new_data.apply(
    lambda row: haversine(row['Latitude'], row['Longitude'], cbd_coordinates[0], cbd_coordinates[1]),
    axis=1
)



# Now use the pipeline to make predictions
new_predictions = pipeline.predict(new_data)

print(new_predictions)
print(new_data.columns)


           id Tranc_YearMonth         town block        street_name  \
9399   140701      2014-10-01    SERANGOON   238    SERANGOON AVE 2   
6365   151836      2012-10-01   QUEENSTOWN    19         QUEEN'S CL   
14189   50808      2020-08-01  BUKIT BATOK   363     BT BATOK ST 31   
11680   73309      2015-11-01      PUNGGOL   135    EDGEDALE PLAINS   
5066   175868      2020-06-01  JURONG WEST   410  JURONG WEST ST 42   
4814    35069      2015-08-01    TOA PAYOH     2    LOR 7 TOA PAYOH   
12465    5069      2019-08-01     TAMPINES  522A    TAMPINES CTRL 7   
10866  110172      2013-10-01    WOODLANDS   541    WOODLANDS DR 16   
9183   164740      2013-04-01     SENGKANG  262D     COMPASSVALE ST   
14389   62742      2020-09-01    WOODLANDS   163    WOODLANDS ST 13   

      storey_range  floor_area_sqm         flat_model  lease_commence_date  \
9399      04 TO 06           104.0            Model A                 1985   
6365      07 TO 09           131.0            Model A         

In [11]:
from joblib import dump

# Assuming your pipeline is named `pipeline`
dump(pipeline, 'housing_price_prediction_pipeline.joblib')


['housing_price_prediction_pipeline.joblib']